In [ ]:
!pip install torch==2.8.0 torchaudio==2.8.0
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 148.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.4/322.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39

In [ ]:
!pip install einops soundfile datasets transformers accelerate librosa

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
# it is necessary to do this for the next pip install to work
!pip install pip==24.0.0
!pip install soundstorm-pytorch
!pip uninstall fairseq
!pip install git+https://github.com/Tps-F/fairseq.git@main

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 56.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 106.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.

In [ ]:
import torchaudio
import os
import torchaudio.transforms as T
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchaudio import functional as AF
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pathlib import Path

In [ ]:
print("PyTorch version:", torch.__version__)

PyTorch version: 2.9.0+cu126


In [ ]:
path_50 = '/content/drive/MyDrive/CIS7000_project/ESC-50-master/audio'
csv_path_50 = '/content/drive/MyDrive/CIS7000_project/ESC-50-master/meta/esc50.csv'
path_8k = '/content/drive/MyDrive/CIS7000_project/UrbanSound8K/audio'
csv_path_8k = '/content/drive/MyDrive/CIS7000_project/UrbanSound8K/metadata/UrbanSound8K.csv'
audios = os.listdir(path_50)
device = "cuda" if torch.cuda.is_available() else "cpu"


k_fold_list = []
k_fold_prefixes = []
k_fold_dirs = os.listdir(path_8k)
for dir in k_fold_dirs:
  def prefix(s):
    return dir + '/' + s
  filenames = os.listdir(os.path.join(path_8k, dir))
  k_fold_list.append(filenames)
  k_fold_prefixes.append(list(map(prefix, filenames)))

# frequency is 48000 for urban8k and 44100 for esc50

'\nsample, frequency = torchaudio.load(path_8k + \'/\' + k_fold_prefixes[0][14])\nprint(sample[0].shape) # why does this sample have two audio tensors?\nprint(frequency)\n\nsample, frequency = torchaudio.load(path_50 + \'/\' + audios[10])\nprint(frequency)\n\nspectrogram = T.Spectrogram(n_fft = 512)\nspec = spectrogram(sample)\n\nfig, axs = plt.subplots(2, 1)\nplot_waveform(sample, frequency, title="Original waveform", ax=axs[0])\nplot_spectrogram(spec[0], title="spectrogram", ax=axs[1])\nfig.tight_layout()\n'

In [ ]:
class AudioDataset_ESC50(Dataset):
  def __init__(self, path, csv, indices = None, val_set = False):
    super().__init__()
    self.path = path
    self.df = pd.read_csv(csv)
    self.index = len(self.df.index) if indices is None else indices
    self.audios = list(self.df['filename'])[-self.index:] if val_set else list(self.df['filename'])[:self.index]
    self.labels = list(self.df['target'])[-self.index:] if val_set else list(self.df['target'])[:self.index]
    self.encoder = OneHotEncoder(sparse_output=False)
    self.frequency = torchaudio.load(self.path + '/' + self.audios[0])[1]

  def __len__(self):
    return len(self.audios)

  def __getitem__(self, idx):
    audio = torchaudio.load(self.path + '/' + self.audios[idx])[0][0]
    label = self.labels[idx]
    return audio, label

  def resample(self, waveform, sample_rate):
      return replace(
          self,
          sample_rate=sample_rate,
          waveform=AF.resample(
              waveform,
              orig_freq=self.frequency,
              new_freq=sample_rate,
          )
      )

class AudioDataset_ESC50test(Dataset):
  def __init__(self, path, csv, indices = None, val_set = False):
    super().__init__()
    self.path = path
    self.df = pd.read_csv(csv)
    self.index = len(self.df.index) if indices is None else indices
    self.audios = list(self.df['filename'])[-self.index:] if val_set else list(self.df['filename'])[:self.index]
    self.encoder_acoustic = AudioToken(tokenizer=Tokenizers.acoustic, device=device)
    self.encoder_semantic = AudioToken(tokenizer=Tokenizers.semantic_s, device=device)
    self.frequency = torchaudio.load(self.path + '/' + self.audios[0])[1]

  def __len__(self):
    return len(self.audios)

  def __getitem__(self, idx):
    audio_path = self.path + '/' + self.audios[idx]
    semantic_tokens = self.encoder_semantic.encode(Path(audio_path))
    acoustic_tokens = self.encoder_acoustic.encode(Path(audio_path))

    return semantic_tokens, acoustic_tokens

class AudioDataset_8K(Dataset):
  def __init__(self, path, csv, kfolds, val, index):
    super().__init__()
    self.path = path
    self.dataframe = pd.read_csv(csv)
    self.index = index
    self.kfolds = kfolds
    self.audios = self.kfolds[index] if val else [ele for i in range(10) for ele in self.kfolds[i] if i != index]
    self.encoder = OneHotEncoder(sparse_output=False)
    self.frequency = torchaudio.load(self.path + '/' + self.audios[0])[1]

  def __len__(self):
    return len(self.audios)

  def __getitem__(self, idx):
    audio = torchaudio.load(self.path + '/' + self.audios[idx])[0][0]
    audio = F.pad(audio, (0, 192000 - audio.size()[0]), "constant", 0)
    label = audio # not currently used
    return audio, label

  def resample(self, waveform, sample_rate):
      return replace(
          self,
          sample_rate=sample_rate,
          waveform=AF.resample(
              waveform,
              orig_freq=self.frequency,
              new_freq=sample_rate,
          )
      )

In [ ]:
train_set = AudioDataset_ESC50(path_50, csv_path_50, 1600)
train_loader = DataLoader(train_set, batch_size = 8, shuffle = True)
val_set = AudioDataset_ESC50(path_50, csv_path_50, 400, True)
val_loader = DataLoader(val_set, batch_size = 8, shuffle = True)

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

In [ ]:
from soundstorm_pytorch import SoundStorm, ConformerWrapper, Conformer, SoundStream, SoundStormTrainer
from audiolm_pytorch import SoundStreamTrainer, EncodecWrapper
import beartype

In [ ]:
beartype.beartype = lambda func: func

In [ ]:
class EncodecForSoundStorm():
    def __init__(self):
        self.encodec = EncodecWrapper()

        self.codebook_size = 1024
        self.num_quantizers = 8
        self.rq_num_quantizers = 8
        self.rq_groups = 1

    def __call__(self, *args, **kwargs):
        return self.encodec(*args, **kwargs)

    def encode(self, *args, **kwargs):
        return self.encodec.encode(*args, **kwargs)

    def decode(self, *args, **kwargs):
        return self.encodec.decode(*args, **kwargs)

    def __getattr__(self, name):
        return getattr(self.encodec, name)

    def to(self, *args, **kwargs):
        self.encodec = self.encodec.to(*args, **kwargs)
        return self

    def train(self, mode=True):
        self.encodec.train(mode)
        return self

    def eval(self):
        self.encodec.eval()
        return self

In [ ]:
def train_one_epoch(model, train_dataloader, optimizer, device, batches):
    running_loss = 0

    for i, data in enumerate(train_dataloader):
      inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)
      optimizer.zero_grad()

      loss, _ = model(inputs)
      loss.backward()

      optimizer.step()
      running_loss += loss.item()

    last_loss = running_loss / batches

    return last_loss


def train(model, n_batches, train_dataloader, val_dataloader, scheduler, optimizer, device, epochs=20):
  epoch_number = 0
  losses = []
  vlosses = []
  for epoch in range(epochs):
    print('EPOCH {}:'.format(epoch_number + 1))

    model.train(True)
    print(soundstream.training, conformer.training)
    avg_loss = train_one_epoch(model, train_dataloader, optimizer, device, n_batches)

    model.eval()

    running_vloss = 0.0

    with torch.no_grad():
      for i, vdata in enumerate(val_dataloader):
        vinputs, vlabels = vdata
        vinputs, vlabels = vinputs.to(device), vlabels.to(device)
        vloss, _ = model(vinputs)
        # vloss = loss_fn(voutputs, vlabels)
        running_vloss += vloss.item()

    avg_vloss = running_vloss / (i + 1)
    scheduler.step(avg_vloss)
    losses.append(avg_loss)
    vlosses.append(avg_vloss)
    # print('LOSS train {} val {}'.format(avg_loss, avg_vloss))

    epoch_number += 1
  df = pd.DataFrame({'loss': losses, 'val_loss': vlosses})
  df.to_csv('esc50losses.csv', index = False)

def train_kfold(model, n_batches, scheduler, optimizer, device):
  epoch_number = 0
  losses = []
  vlosses = []
  for i in range(10):
    train_set = AudioDataset_8K(path_8k, csv_path_8k, k_fold_prefixes, val=False, index=i)
    val_set = AudioDataset_8K(path_8k, csv_path_8k, k_fold_prefixes, val=True, index=i)
    train_dataloader = DataLoader(train_set, batch_size = 16, shuffle = False)
    val_dataloader = DataLoader(val_set, batch_size = 16, shuffle = False)
    print('EPOCH {}:'.format(epoch_number + 1))

    model.train(True)
    avg_loss = train_one_epoch(model, train_dataloader, optimizer, device, n_batches)

    model.eval()

    running_vloss = 0.0

    with torch.no_grad():
      for i, vdata in enumerate(val_dataloader):
        vinputs, vlabels = vdata
        vinputs, vlabels = vinputs.to(device), vlabels.to(device)
        vloss, _ = model(vinputs)
        running_vloss += vloss.item()

    avg_vloss = running_vloss / (i + 1)
    scheduler.step(avg_vloss)
    losses.append(avg_loss)
    vlosses.append(avg_vloss)
    # print('LOSS train {} val {}'.format(avg_loss, avg_vloss))

    epoch_number += 1
  df = pd.DataFrame({'loss': losses, 'val_loss': vlosses})
  df.to_csv('urban8klosses.csv', index = False)

def plot_losses(csv_path):
  df = pd.read_csv(csv_path)
  loss = df['loss']
  val_loss = df['val_loss']
  plt.plot(loss, label = 'Train loss')
  plt.plot(val_loss, label = 'Validation loss')
  plt.title('Losses over Epochs')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# conformer and soundstream hyperparams from https://github.com/lucidrains/soundstorm-pytorch

conformer = ConformerWrapper(
    codebook_size = 1024,
    num_quantizers = 8,
    conformer = dict(
        dim = 512,
        depth = 2
    ),
)

soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 12,
    attn_window_size = 128,
    attn_depth = 2
)

# enc = EncodecModel.encodec_model_24khz()
enc = EncodecForSoundStorm()
enc.eval()
# trainer = SoundStreamTrainer(soundstream, folder = path_50, batch_size = 4, grad_accum_every = 2, data_max_length_seconds = 5, num_train_steps = 200).to(device)
# trainer.train()

model = SoundStorm(conformer, soundstream = enc).to(device)
# trainer = SoundStormTrainer(model, num_train_steps = 20, num_warmup_steps = 2, batch_size = 8, dataset = train_set)
# trainer.train() # okay i don't know if I can make this trainer work

optimizer = optim.Adam(model.parameters(), lr = 0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 4)

train(model, 200, train_loader, val_loader, scheduler, optimizer, device, epochs=20)
# train_kfold(model, 245, loss_fn, scheduler, optimizer, device)
# audio = torch.randn(2, 10080).to(device)
# loss, _ = model(audio)
# loss.backward()

generated_audio = model.generate(seconds = 10, batch_size = 2)  # generate 10 seconds of audio (it will calculate the length in seconds based off the sampling frequency and cumulative downsamples in the soundstream passed in above)
save_path = "generated_esc50.wav" # change based on source
generated_audio = generated_audio.cpu()
torchaudio.save(save_path, generated_audio, sample_rate = 88200) # sample rate at least 88200 for esc50, at least 96000 for urban8k

BeartypeCallHintParamViolation: Method soundstorm_pytorch.soundstorm.SoundStorm.__init__() parameter soundstream=<__main__.EncodecForSoundStorm object at 0x7ec472be0ad0> violates type hint audiolm_pytorch.soundstream.SoundStream | None, as <class "__main__.EncodecForSoundStorm"> <__main__.EncodecForSoundStorm object at 0x7ec472be0ad0> not <class "builtins.NoneType"> or <class "audiolm_pytorch.soundstream.SoundStream">.

In [ ]:
from google.colab import runtime

runtime.unassign()

In [ ]:
!nvidia-smi